In [31]:
import pandas as pd
import json
import numpy as np

def flatten_data(data):
    flattened_data = []
    for meal in data:
        for dish in meal["dishes"]:
            row_data = {
                "Meal": meal["meal"],
                "dish_name": dish["name"],
                "calories": None,
                "carbs": None,
                "fat": None,
                "protein": None,
                "sodium": None,
                "sugar": None
            }
            for nutrition in dish["nutritions"]:
                row_data[nutrition["name"].lower()] = nutrition["value"]
            flattened_data.append(row_data)

    df = pd.DataFrame(flattened_data)
    return df

def flatten_total_data(data, ID_Number, Date, Meal):
    flattened_data = []
    row_data = {
        "ID_Number": ID_Number,
        "Date": Date,
        "Meal": Meal,
        "Total_Calories": None,
        "Total_Carbs": None,
        "Total_Fat": None,
        "Total_Protein": None,
        "Total_Sodium": None,
        "Total_Sugar": None,
        "Goal_Calories": None,
        "Goal_Carbs": None,
        "Goal_Fat": None,
        "Goal_Protein": None,
        "Goal_Sodium": None,
        "Goal_Sugar": None
    }

    for nutrition in data['total']:
        row_data[f"Total_{nutrition['name']}"] = nutrition['value']
    for nutrition in data['goal']:
        row_data[f"Goal_{nutrition['name']}"] = nutrition['value']

    flattened_data.append(row_data)
    return flattened_data


# Read the Excel file without header
file_path = 'My_Fitness_Pal_Data_2.xlsx'
try:
    xf = pd.read_excel(file_path, header=None)
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
    exit()
except pd.errors.EmptyDataError:
    print(f"File '{file_path}' is empty.")
    exit()

# Create empty lists to store the rows
meal_data_list = []  # List to store meal data DataFrames
total_data_list = []  # List to store total data DataFrames

# Iterate through all rows
for index, row in xf.iterrows():
    try:
        data = json.loads(row[2])
    except json.JSONDecodeError:
        print(f"Error decoding JSON data in row {index}. Skipping.")
        continue
    df_meals = flatten_data(data)
    df_meals.insert(0, 'ID_Number', row[0])
    df_meals.insert(1, 'Date', row[1])
    meal_data_list.append(df_meals)

    total_data = json.loads(row[3])  # Assuming the total_data is in the 4th column
    df_total_data = pd.DataFrame(flatten_total_data(total_data, row[0], row[1], "MY food"))
    total_data_list.append(df_total_data)

In [32]:
# Concatenate the meal data DataFrames into a single DataFrame
meal_data_df = pd.concat(meal_data_list, ignore_index=True)

# Concatenate the total data DataFrames into a single DataFrame
total_data_df = pd.concat(total_data_list, ignore_index=True)

# Drop duplicate entries based on 'ID_Number' and 'Date' for both DataFrames
meal_data_df = meal_data_df.drop_duplicates(subset=['ID_Number', 'Date', 'Meal', 'dish_name'])
total_data_df = total_data_df.drop_duplicates(subset=['ID_Number', 'Date', 'Meal'])

# Print the resulting DataFrames
#display(meal_data_df)

# Create CSV
#meal_data_df.to_csv('Meal_Data_Frame.csv', index=False)

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.linear_model import LinearRegression
from scipy.signal import find_peaks

# Creating an empty DataFrame for the calculated statistics
columns = ['Person ID', 'Start Date', 'End Date', 'Days Logged']
stats_df = pd.DataFrame(columns=columns)

manual_goal_no_change = []
manual_goal_change = []
automatic_goal_change = []
automatic_goal_no_change = []
automatic_goal_change_pos = []
automatic_goal_change_neg = []

# Iterate through unique ID numbers in 'total_data_df'
with PdfPages('Fitness_Pal_Data_Figures.pdf') as pdf:
    for person_id in total_data_df['ID_Number'].unique():
        # Check if the maximum number of ID numbers has been reached

        # Filter data for the current person_id
        person_data = total_data_df[total_data_df['ID_Number'] == person_id]

        # Initialize variables to store calculated statistics
        start_date = person_data['Date'].min()
        end_date = person_data['Date'].max()
        unique_dates = person_data['Date'].unique()
        days_active = len(unique_dates)

        # Calculate the differences between consecutive dates
        date_diffs = (person_data['Date'] - person_data['Date'].shift()).dropna()

        # Calculate average days between entry
        avg_days_between_entry = np.round(date_diffs.mean().days) if not date_diffs.empty else 0
        std_days_between_entry = np.round(date_diffs.std().days) if not date_diffs.empty else 0
        median_days_between_entry = np.round(date_diffs.median().days) if not date_diffs.empty else 0

        # Calculate longest hiatus
        longest_hiatus = np.round(date_diffs.max().days) if not date_diffs.empty else 0

        # Filter data for the current person_id
        person_data_meal = meal_data_df[meal_data_df['ID_Number'] == person_id]

        # Calculate average meals a day
        avg_meals_per_day = person_data_meal.groupby('Date')['Meal'].nunique().mean()
        std_meals_per_day = person_data_meal.groupby('Date')['Meal'].nunique().std()
        median_meals_per_day=  person_data_meal.groupby('Date')['Meal'].nunique().median()

        # Number of Days Logged
        days_logged = (end_date - start_date).days + 1
        days_ratio = days_active/days_logged

        # Calorie goals:
        avg_calorie_goal = round((person_data['Goal_Calories'].mean()))
        std_calorie_goal = round((person_data['Goal_Calories'].std(skipna=True))) if len(person_data['Goal_Calories'].dropna().unique()) > 1 else 0
        median_calorie_goal = round((person_data['Goal_Calories'].median()))
        avg_calories_consumed = round(person_data['Total_Calories'].mean()) if not np.isnan(person_data['Total_Calories'].mean()) else 0
        std_calories_consumed = round(person_data['Total_Calories'].std(skipna=True)) if len(person_data['Total_Calories'].dropna().unique()) > 1 else 0
        median_calories_consumed = round(person_data['Total_Calories'].median()) if not np.isnan(person_data['Total_Calories'].median()) else 0
        # Append the calculated statistics to the stats_df
        stats_df = pd.concat([stats_df, pd.DataFrame({
            'Person ID': [person_id],
            'Start Date': [start_date],
            'End Date': [end_date],
            'Days Active': [days_active],
            'Days active to span Ratio':[days_ratio],
            'Days Logged': [days_logged],
            'Average Days Between Entry': [avg_days_between_entry],
            'Median Days Between Entry': [median_days_between_entry],
            'Standard Deviation Days Betweeen Entry': [std_days_between_entry],
            'Longest Hiatus (Days)': [longest_hiatus],
            'Average Meals Per Day': [avg_meals_per_day],
            'Median Meals Per Day': [median_meals_per_day],
            'Standard Deviation Meals': [std_meals_per_day],
            'Average Calorie Goal': [avg_calorie_goal],
            'Standard Deviation Calorie Goal': [std_calorie_goal],
            'Median Calorie Goal': [median_calorie_goal],
            'Average Calories Consumed per Day': [avg_calories_consumed],
            'Standard Deviation Calories Consumed': [std_calories_consumed],
            'Median Calories Consumed per Day': [median_calories_consumed]
        })], ignore_index=True)

        # Plotting Each Time Series of Calorie Goals
        #plt.figure()
        #plt.plot(person_data['Date'], person_data['Goal_Calories'], marker='o', linestyle='-')  # Plot calorie goals over dates
        #plt.axhline(person_data['Goal_Calories'].mean(), color='green', linestyle='dashed', linewidth=1, label= 'Goal Calories Mean')
        #plt.axhline(person_data['Goal_Calories'].median(), color='red', linestyle='dashed', linewidth=1, label= 'Goal Calories Median')
        #plt.title(f'Changes in Caloric Goals Person {person_id}')  
        #plt.xlabel('Date')  
        #plt.ylabel('Calorie Goal')  
        #plt.xticks(rotation=45)  
        #plt.tight_layout() 
        #plt.legend() 
        #pdf.savefig()  
        #plt.close() 
        
        
        # Manual Goal with no change
        if np.std(person_data['Goal_Calories']) == 0: 
            manual_goal_no_change.append(person_id)
            
      
        # Manual Goal with Change
        
        def count_stable_periods_and_points(data, threshold=10, min_length_percent=0.1):
            data = data.dropna()
            min_length = round(len(data) * min_length_percent)
            num_stable_periods = 0
            start = 0
            stable_indicies = []

            while start < len(data):
                end = start
                while end < len(data) - 1 and abs(data.iloc[end + 1] - data.iloc[start]) <= threshold:
                    end += 1
                if end - start + 1 >= min_length:
                    num_stable_periods += 1
                    stable_indicies.extend(range(start, end + 1))
                start = end + 1

            return num_stable_periods,stable_indicies

        num_stable_periods, stable_indicies = count_stable_periods_and_points(person_data['Goal_Calories'])
        stable_percent = len(stable_indicies)/len(person_data['Goal_Calories'])
        
        if person_id not in manual_goal_no_change:
            if num_stable_periods >= 2 and stable_percent > 0.5: 
                manual_goal_change.append(person_id)
            if num_stable_periods == 1 and stable_percent > .75:
                manual_goal_change.append(person_id)
        
        
        # Automatic Goal with Change 
        
        
        non_na_goal_calories = person_data['Goal_Calories'].dropna()
        minima_indices = find_peaks(-non_na_goal_calories)[0]
        local_minima_list = non_na_goal_calories.iloc[minima_indices].tolist()
        
        percentage_change = np.diff(local_minima_list)/local_minima_list[:-1]*100
        
        exceed_threshold = np.abs(percentage_change) > 10 
        
        if any(exceed_threshold) and person_id not in manual_goal_change: 
            automatic_goal_change.append(person_id)
        
        # Calorie goal Patterns Automatic with change
        
        if person_id not in manual_goal_no_change and person_id not in automatic_goal_change and person_id not in manual_goal_change: 
            automatic_goal_no_change.append(person_id)

            

    
print(manual_goal_change)
print(len(manual_goal_change))
print(manual_goal_no_change)
print(len(manual_goal_no_change))
print(automatic_goal_no_change)
print(len(automatic_goal_no_change))
print(automatic_goal_change)
print(len(automatic_goal_change))


# Print the resulting DataFrame with calculated statistics
#display(stats_df)

# Create CSV
stats_df.to_csv('Fitness_Data_Analysis_2.csv', index=False)


file_path = 'manual_goal_change_list_2.txt'
with open(file_path, 'w') as file: 
    for person_id in manual_goal_change:
        file.write(f"{person_id}\n")
    file.write(f"Total Number of IDs: {len(manual_goal_change)}\n")
    file.write(f"Percentage of total: {len(manual_goal_change)/len(total_data_df['ID_Number'].unique())}")
print(f"IDS have been written to {file_path}")
               
file_path = 'manual_goal_no_change_list_2.txt'
with open(file_path, 'w') as file: 
    for person_id in manual_goal_no_change:
        file.write(f"{person_id}\n")
        
    file.write(f"Total Number of IDs: {len(manual_goal_no_change)}\n")
    file.write(f"Percentage of total: {len(manual_goal_no_change)/len(total_data_df['ID_Number'].unique())}")
print(f"IDS have been written to {file_path}")
               
file_path = 'automatic_goal_no_change_list_2.txt'
with open(file_path, 'w') as file: 
    for person_id in automatic_goal_no_change:
        file.write(f"{person_id}\n")
    file.write(f"Total Number of IDs: {len(automatic_goal_no_change)}\n")
    file.write(f"Percentage of total: {len(automatic_goal_no_change)/len(total_data_df['ID_Number'].unique())}")
print(f"IDS have been written to {file_path}")
               
file_path = 'automatic_goal_change_list_2.txt'
with open(file_path, 'w') as file: 
    for person_id in automatic_goal_change:
        file.write(f"{person_id}\n")
    file.write(f"Total Number of IDs: {len(automatic_goal_change)}\n")
    file.write(f"Percentage of total: {len(automatic_goal_change)/len(total_data_df['ID_Number'].unique())}")
print(f"IDS have been written to {file_path}")



C:\Users\natal\AppData\Local\Temp\ipykernel_16012\2372123832.py:143: RuntimeWarning: divide by zero encountered in divide
  percentage_change = np.diff(local_minima_list)/local_minima_list[:-1]*100


[1015, 1017, 1018, 1022, 1025, 1028, 1029, 1034, 1035, 1036, 1042, 1046, 1047, 1048, 1049, 1064, 1065, 1070, 1072, 1073, 1074, 1076, 1077, 1079, 1083, 1084, 1086, 1087, 1088, 1089, 1091, 1097, 1099, 1101, 1102, 1103, 1104, 1106, 1111, 1114, 1117, 1121, 1124, 1125, 1129, 1130, 1133, 1138, 1139, 1145, 1148, 1149, 1156, 1157, 1160, 1164, 1169, 1171, 1172, 1173, 1175, 1176, 1179, 1181, 1182, 1183, 1185, 1186, 1187, 1188, 1192, 1193, 1198, 1200, 1204, 1205, 1208, 1210, 1214, 1215, 1218, 1229, 1232, 1233, 1238, 1241, 1249, 1252, 1253, 1255, 1263, 1264, 1268, 1269, 1276, 1278, 1282, 1285, 1288, 1289, 1290, 1292, 1293, 1295, 1296, 1297, 1299, 1303, 1307, 1311, 1324, 1326, 1329, 1333, 1334, 1335, 1336, 1338, 1339, 1342, 1344, 1356, 1357, 1365, 1366, 1367, 1368, 1372, 1374, 1375, 1379, 1385, 1388, 1392, 1393, 1398, 1403, 1404, 1406, 1409, 1418, 1422, 1424, 1425, 1428, 1430, 1434, 1435, 1436, 1439, 1442, 1444, 1446, 1450, 1451, 1453, 1454, 1458, 1460, 1462, 1464, 1468, 1470, 1471, 1474, 1479, 148